<a href="https://colab.research.google.com/github/marlon-br/glow-tts-colab/blob/master/MozillaWavsToText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install deepspeech==0.6.0
!apt -qq install -y sox

The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3
Suggested packages:
  file libsox-fmt-all
The following NEW packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3 sox
0 upgraded, 8 newly installed, 0 to remove and 43 not upgraded.
Need to get 760 kB of archives.
After this operation, 6,717 kB of additional disk space will be used.
Selecting previously unselected package libopencore-amrnb0:amd64.
(Reading database ... 144328 files and directories currently installed.)
Preparing to unpack .../0-libopencore-amrnb0_0.1.3-2.1_amd64.deb ...
Unpacking libopencore-amrnb0:amd64 (0.1.3-2.1) ...
Selecting previously unselected package libopencore-amrwb0:amd64.
Preparing to unpack

In [2]:
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.6.0/deepspeech-0.6.0-models.tar.gz
!tar -xvzf deepspeech-0.6.0-models.tar.gz
!ls -l ./deepspeech-0.6.0-models/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   641  100   641    0     0    907      0 --:--:-- --:--:-- --:--:--   906
100 1172M  100 1172M    0     0  15.6M      0  0:01:15  0:01:15 --:--:-- 16.3M
deepspeech-0.6.0-models/
deepspeech-0.6.0-models/lm.binary
deepspeech-0.6.0-models/output_graph.pbmm
deepspeech-0.6.0-models/output_graph.pb
deepspeech-0.6.0-models/trie
deepspeech-0.6.0-models/output_graph.tflite
total 1350664
-rw-r--r-- 1 501 staff 945699324 Dec  3  2019 lm.binary
-rw-r--r-- 1 501 staff 188914896 Dec  3  2019 output_graph.pb
-rw-r--r-- 1 501 staff 188915850 Dec  3  2019 output_graph.pbmm
-rw-r--r-- 1 501 staff  47335752 Dec  3  2019 output_graph.tflite
-rw-r--r-- 1 501 staff  12200736 Dec  3  2019 trie


In [8]:
!deepspeech --model deepspeech-0.6.0-models/output_graph.pb --lm deepspeech-0.6.0-models/lm.binary --trie \
           ./deepspeech-0.6.0-models/trie --audio "/content/drive/My Drive/wavs-vctk/speakers/p229/p229_001.wav"

Loading model from file deepspeech-0.6.0-models/output_graph.pb
TensorFlow: v1.14.0-21-ge77504a
DeepSpeech: v0.6.0-0-g6d43e21
2020-06-18 08:45:06.631316: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Loaded model in 0.187s.
Loading language model from files deepspeech-0.6.0-models/lm.binary ./deepspeech-0.6.0-models/trie
Loaded language model in 0.000243s.
Running inference.
please cause della
Inference took 2.564s for 6.407s audio file.


In [9]:
import deepspeech

model_file_path = 'deepspeech-0.6.0-models/output_graph.pbmm'
beam_width = 500
model = deepspeech.Model(model_file_path, beam_width)

# Add language model for better accuracy
lm_file_path = 'deepspeech-0.6.0-models/lm.binary'
trie_file_path = 'deepspeech-0.6.0-models/trie'
lm_alpha = 0.75
lm_beta = 1.85
model.enableDecoderWithLM(lm_file_path, trie_file_path, lm_alpha, lm_beta)

0

In [21]:
from typing import Tuple
import wave
import sox
import numpy as np

temp_filename = '/content/output.wav'
tfm = sox.Transformer()
tfm.set_output_format(rate=16000)

def read_wav_file(filename) -> Tuple[bytes, int]:

    sample_rate = sox.file_info.sample_rate(filename)

    if (sample_rate != 16000):
        print("resampling")

        
        status = tfm.build(input_filepath=filename,
                            output_filepath=temp_filename)
        
        filename = temp_filename

    with wave.open(filename, 'rb') as w:
        rate = w.getframerate()
        frames = w.getnframes()
        buffer = w.readframes(frames)

    return buffer, rate

In [22]:
import numpy as np
import os


!rm -r /content/results
!mkdir /content/results

files_dir = "p229"
full_files_path = os.path.join("/content/drive/My Drive/wavs-vctk/speakers", files_dir)

new_filename = os.path.join("/content/results", "speaker_p229.csv")

file = open(new_filename,"w")


for i, filename in enumerate(os.listdir(full_files_path)[:3]):
    print(filename)
    full_path = os.path.join(full_files_path, filename)


    buffer, rate = read_wav_file(full_path)
    data16 = np.frombuffer(buffer, dtype=np.int16)

    output_string = model.stt(data16)

    new_item = str(i) + "," + os.path.join(files_dir, filename) + "," + output_string + "~\n"
    print(new_item)      
    file.write(new_item)

file.close() 

output_file: /content/output.wav already exists and will be overwritten on build


p229_004.wav
resampling


output_file: /content/output.wav already exists and will be overwritten on build


0,p229/p229_004.wav,we also need a small plastic snake and a big toy frog for the kids~

p229_006.wav
resampling


output_file: /content/output.wav already exists and will be overwritten on build


1,p229/p229_006.wav,when the sunlight strikes range up in the air they act as a prism and former rainbow~

p229_002.wav
resampling
2,p229/p229_002.wav,as had to bring these things with her from the store~



In [ ]:
import numpy as np

def deepspeech_batch_stt(filename: str, lang: str, encoding: str) -> str:
    buffer, rate = read_wav_file(filename)
    data16 = np.frombuffer(buffer, dtype=np.int16)
    return model.stt(data16)

# Run tests
for t in TESTCASES:
    print('\naudio file="{0}"    expected text="{1}"'.format(t['filename'], t['text']))
    print('deepspeech-batch-stt: "{}"'.format(
        deepspeech_batch_stt(t['filename'], t['lang'], t['encoding'])
    ))